# Proyecto completo en Javascript

Una vez vistos todos los contenidos de la parte de Javascript, estamos listos para hacer un proyecto entero. No nos detendremos a explicar el porqué de los pasos a seguir, tan solo a hacer referencia a los capítulos anteriores. 

[Video de proyecto en JS](https://youtu.be/gFOOsefQMlg)

> El orden en que se realizan los pasos de este proyecto puede ser alterado. De hecho, el orden es un poco arbitrario porque es el que podriamos seguir en clase, no en un proyecto real. En clase se puede ir mejorando incrementalmente y hacer i deshacer. En un proyecto real comenzariamos instalando todas las librerias, configurando los tests, linters, hooks, github actions i el despliegue automatizado antes de hacer la primera línea de código para ser más eficientes. También empezaríamos directamente con programación funcional y reactiva. Pero puede que todo ese entorno resulte demasiado denso para el alumnado que empieza. Así que comenzaremos con una configuración mínima y refactorizaremos en algunas ocasiones. 

## Configuración inicial

En realidad, para empezar a hacer algo en Javascript en el frontend, tan solo necesitamos un archivo HTML y uno JS. Se puede empezar por ahí e ir incorporando las distintas librerias, bundlers y demás. También se puede empezar con el esqueleto del CI/CD completo y luego empezar a programar. Empezaremos de una manera "mixta", configurando `Vite` y los archivos y servicios que nos afectan a la programación desde el principio e iremos instalando y configurando conforme sea necesario. 

### Vite

Ejecutamos los comandos básicos:

```bash
npm create vite@latest my-app -- --template vanilla
cd my-app
npm install
npm run dev
```

Con esto ya tenemos una plantilla con la que comenzar. Eliminaremos el código de ejemplo, las imágenes y el css de ejemplo para empezar desde cero. Este puede ser un buen momento para instalar algun framework de estilos como `Bootstrap`. En nuesto caso, como no nos interesa tanto la parte visual, vamos a instalarlo y así simplificamos ese apartado:

https://getbootstrap.com/docs/5.2/getting-started/vite/ 


Podemos crear una estructura html mínima para el `index.html` a partir de la plantilla proporcionada por Vite:

```html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Vite App</title>
  </head>
  <body>
    <div id="app">
      <div id="menu"></div>
      <div id="container" class="container"></div>
    </div>
    <script type="module" src="/main.js"></script>
  </body>
</html>
```

Puesto que ya tenemos `Bootstrap`, crearemos el menú más simple, un `navbar` en views/views.js:

```javascript
const buildMenu = () => {
  const divWrapper = document.createElement("div");
  const menu = `<nav class="navbar navbar-expand-lg bg-light">
  <div class="container-fluid">
    <a class="navbar-brand" href="#">Movies 2024</a>
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse" id="navbarSupportedContent">
      <ul class="navbar-nav me-auto mb-2 mb-lg-0">
        <li class="nav-item">
          <a class="nav-link active" aria-current="page" href="#">Home</a>
        </li>
        <li class="nav-item">
          <a class="nav-link" href="#">Link</a>
        </li>
        <li class="nav-item dropdown">
          <a class="nav-link dropdown-toggle" href="#" role="button" data-bs-toggle="dropdown" aria-expanded="false">
            Dropdown
          </a>
          <ul class="dropdown-menu">
            <li><a class="dropdown-item" href="#">Action</a></li>
            <li><a class="dropdown-item" href="#">Another action</a></li>
            <li><hr class="dropdown-divider"></li>
            <li><a class="dropdown-item" href="#">Something else here</a></li>
          </ul>
        </li>
        <li class="nav-item">
          <a class="nav-link disabled">Disabled</a>
        </li>
      </ul>
      <form class="d-flex" role="search">
        <input class="form-control me-2" type="search" placeholder="Search" aria-label="Search">
        <button class="btn btn-outline-success" type="submit">Search</button>
      </form>
    </div>
  </div>
</nav>`;
  divWrapper.innerHTML = menu;
  return divWrapper.querySelector("nav");
};
```

y en main.js:

```javascript
import "./styles.scss";
//import * as bootstrap from "bootstrap";
import { buildMenu } from "./views/views";

document.addEventListener("DOMContentLoaded", async () => {
  const menuDiv = document.querySelector("#menu");
  const containerDiv = document.querySelector("#container");
  containerDiv.innerHTML = "";
  menuDiv.append(buildMenu());
});
```

### Vitest

Durante el proyecto vamos a seguir una metodología próxima a TDD. Por tanto, desde el principio necesitamos instalar los test. 

```bash
npm install -D vitest
```

Debemos crear archivos de prueba que tengan la extensión `.test.js`.

En package.json:

```json
{
  "scripts": {
    "test": "vitest",
    "coverage": "vitest run --coverage"
  }
}
```

```bash
npm run test
npm run coverage
npx vitest --ui
```

En el artículo de tests tenemos lo anterior explicado con detalle.

## Estructura del proyecto

El proyecto va a ser muy básico, así que no necesitamos una gran arquitectura. Separaremos la conexión al backend de la vista y los controladores. Por tanto, inicialmente podemos crear carpetas para `modelos` o `servicios` y para las `vistas`. Los modelos se conectarán a la base de datos y servirán los datos de forma reactiva, es decir, con RxJS. Las vistas crearán elementos del `DOM` mediante template literals y serán reactivas. Haremos un CRUD con autenticación en el que el backend será `Supabase`. 

## Backend básico con Supabase

Nos damos de alta en Supabase y creamos un proyecto nuevo. Una vez creado, creamos las tablas de la base de datos y ponemos algunos datos. En nuestro caso, he creado inicialmente una tabla `movies` con el contenido de una base de datos de unas 45000 películas de la IMDB. Probamos el comando que nos recomienda para ver las películas: 

```bash
curl 'https://ygvtpucoxveebizknhat.supabase.co/rest/v1/movies?select=*' \
-H "apikey: SUPABASE_KEY" \
-H "Authorization: Bearer SUPABASE_KEY"
```

Lo podemos probar también en Postman. Esta información proporcionada por el `API Docs` de Supabase será la base para una función inicial de obtención de las películas:

```javascript
const getSupabase = async (table) => {
  try {
    let response = await fetch(
      `https://ygvtpucoxveebizknhat.supabase.co/rest/v1/${table}?select=*`,
      {
        headers: {
          apikey,
          Authorization: `Bearer ${apikey}`,
        },
      },
    );
    if (!response.ok) {
      return Promise.reject("Bad request");
    }
    return response;
  } catch {
    return Promise.reject("Network Error");
  }
};

const getData = (response) => {
  return response.json();
};

```

Como esta función, podemos ir haciendo las demás. 

Veamos un objeto de los que retorna esta petición:

```json
{
		"adult": false,
		"belongs_to_collection": "Toy Story Collection",
		"budget": "30000000",
		"original_language": "en",
		"original_title": "Toy Story",
		"overview": "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
		"popularity": 21.946943,
		"release_date": "1995-10-30",
		"revenue": "373554033.0",
		"runtime": "81.0",
		"tagline": "not available",
		"title": "Toy Story",
		"vote_average": "7.7",
		"vote_count": "5415.0",
		"languages": "['English']",
		"day_of_week": "Monday",
		"month": "Oct",
		"season": "Q4",
		"year": "1995",
		"has_homepage": "YES",
		"genre": "['Animation', 'Comedy', 'Family']",
		"companies": "['Pixar Animation Studios']",
		"countries": "['United States of America']",
		"id": "8554eb64-1588-4480-84aa-ab1796b1707b"
}
```

Como se puede ver, es un objeto que tiene un "id" generado automáticamente por Supabase que es lo que lo identifica en la base de datos. Uno de los problemas que tiene es que el género, compañías, idiomas y países son "arrays", pero tratados por Supabase como Strings. Por tanto, si queremos interpretarlos hay que parsearlos. Por lo demás no parece tener más problemas. 

Los Arrays entre comillas que nos retorna el servidor son una buena ocasión para practicar TDD. Este es el test:

```javascript
describe("stringToArray", async () => {
    test("stringToArray should return an Array of movies", () => {
      let complexArray =  `['Procirep', 'Constellation Productions', 'France 3 Cinéma', 'Claudie Ossard Productions', 'Eurimages', 'MEDIA Programme of the European Union', 'Cofimage 5', 'Televisión Española (TVE)', 'Tele München Fernseh Produktionsgesellschaft (TMG)', "Club d'Investissement Média", 'Canal+ España', 'Elías Querejeta Producciones Cinematográficas S.L.', 'Centre National de la Cinématographie (CNC)', 'Victoires Productions', 'Constellation', 'Lumière Pictures', 'Canal+', 'Studio Image', 'Cofimage 4', 'Ossane', 'Phoenix Images']`;
      let result = _views.stringToArray(complexArray);
      expect(result).toBeInstanceOf(Array);
      expect(result.length).toBe(21);
    });
  });
```
Observemos que `"Club d'Investissement Média"` tiene comillas dobles y una comilla simple en medio. Esto complica el JSON.parse.

Y la función resultante:

```javascript
const stringToArray = (string) => {
  let convertedString = string.replace(/',/g, '",')
      .replace(/ '/g, ' "')
      .replace(/\['/g, '["')
      .replace(/'\]/g, '"]');
  console.log(string, convertedString);
  return JSON.parse(convertedString);
};
```

Puesto que tenemos funciones genéricas para obtener datos y funciones específicas de las películas para transformar los datos, es buena idea separarlas. Por eso, podemos crear un fichero para el `modelo` "movies" y otro para las peticiones http. El primero tendrá esas funciones para transformar las películas en un array válido. Por ejemplo:

```javascript
const parseMovies = (movies) => {
    let moviesCopy = structuredClone(movies);
    moviesCopy.forEach(m => { 
    m.genre = stringToArray(m.genre);
    m.companies = stringToArray(m.companies);
    m.countries = stringToArray(m.countries);
  });
  return moviesCopy;
} 
```

Cuyo test es:

```javascript
    test("parseMovies should return an Array of movies with arrays parsed", () => {
        let result = _movies.parseMovies(exampleMovies);
      expect(result).toBeInstanceOf(Array);
      expect(result.length).toBe(4);
      expect(result.every(m=> m.genre instanceof Array)).toBe(true);
      expect(result.every(m=> m.companies instanceof Array)).toBe(true);
      expect(result.every(m=> m.countries instanceof Array)).toBe(true);
    });
```

Luego, para ver las películas de forma básica en la vista, gracias a `Bootstrap` podemos hacer una lista desplegable: 

```javascript
const buildMoviesComponent = (movies) => {
  const divWrapper = document.createElement("div");
  divWrapper.classList.add("accordion");
  divWrapper.innerHTML = movies
    .map(
      (m, index) => `<div class="accordion-item">
   
<h2 class="accordion-header" id="panelsStayOpen-heading${index}">
    <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#panelsStayOpen-collapse${index}" aria-expanded="true" aria-controls="panelsStayOpen-collapse${index}">
    ${m.original_title}
    </button>
</h2>
<div id="panelsStayOpen-collapse${index}"  class="accordion-collapse collapse" aria-labelledby="panelsStayOpen-heading${index}">
<div class="accordion-body">
<h3>Overview</h3>
${m.overview}
<h3>Technical Data:</h3>
<ul class="list-group">
  <li class="list-group-item">Release Date: ${m.release_date}</li>
  <li class="list-group-item">Revenue: ${m.revenue}</li>
  <li class="list-group-item">Runtime: ${m.runtime}</li>
  <li class="list-group-item">Tagline: ${m.tagline}</li>
  <li class="list-group-item">Vote Average: ${m.vote_average}</li>
  <li class="list-group-item">Vote Count: ${m.vote_count}</li>
  <li class="list-group-item">Year: ${m.year}</li>
</ul>
<h3>Genres</h3>
<div class="btn-group" role="group" aria-label="Basic example">
  ${m.genre
    .map(
      (g) =>
        `<button type="button" class="btn btn-primary" data-genre="${g}">${g}</button>`
    )
    .join("")}
</div>
<h3>Companies</h3>
<div class="btn-group" role="group" aria-label="Basic example">
  ${m.companies
    .map(
      (g) =>
        `<button type="button" class="btn btn-primary" data-company="${g}">${g}</button>`
    )
    .join("")}
</div>
<h3>Countries</h3>
<div class="btn-group" role="group" aria-label="Basic example">
  ${m.countries
    .map(
      (g) =>
        `<button type="button" class="btn btn-primary" data-country="${g}">${g}</button>`
    )
    .join("")}
</div>
</div>
</div>

</div>`
    )
    .join("");
  return divWrapper;
};
```

En el ejemplo anterior usamos un `template literal` para cada película i dentro usamos más para los botones. De momento no le hemos dado funcionalidad a los botones. 

```{figure} ./imgs/projecte1.png
---
scale: 50%
align: center
---
Así queda de momento la aplicación 
```

## Búsqueda en el backend

Ahora que ya tenemos la capacidad de descargar cosas del backend, vamos a implementar una búsqueda para ver las posibilidades.

Cuando un usuario pulse uno de los botones de, por ejemplo, el género de una película, se debería buscar en Supabase todas las películas de ese género. Puesto que se trata de una base de datos PostgreSQL, esto será traducido finalmente a una consulta SQL. Pero Supabase proporciona una interfaz API REST para hacer esta petición de forma remota. Tengamos en cuenta también que el género se almacena en una string, aunque parezca un array. 

Para hacer una búsqueda en Supabase usando el API REST, seguiremos la siguente sintaxis:

    products?select=*&asin=like.*AX*
    
Como se ve, `products` es la tabla, después va `select=*` para indicar que necesitamos todas las columnas y a continuación la condición de búsqueda. Imaginemos que en la tabla `movies` necesitamos las películas del género `action`.

    https://ygvtpucoxveebizknhat.supabase.co/rest/v1/movies?genre=like.*Action*&select=*&

Por tanto, hay que construir la petición. Esto va a cambiar las funciones que hemos creado antes para poder hacerlas más genéricas y reutilizar código.

Este es el cambio en la petición:

```javascript
const getSupabase = async (table, columns, search) => {
  try {
    let response = await fetch(
      `https://ygvtpucoxveebizknhat.supabase.co/rest/v1/${table}?select=${columns ? columns : '*'}${search ? `&${search}` : ''}`,
      {
      ....
```

La búsqueda cambiará la lista de películas descargadas que se visualizan. Para volver atrás, una buena manera es implementar un router:

### Router

Este sería el `main.js` aplicando un router:

```javascript
const fillElement = (container) => (content) => { container.innerHTML = ""; container.append(content); }

const router = async (route, container) => {
  const fillContainer = fillElement(container);
  // Rutas con expresiones regulares
  if (/#\/movies\/genre\/.+/.test(route)) {
    let genreID = route.split("/")[3];
    let movies = await getMovies(`genre=ilike.*${genreID}*`);
    fillContainer(buildMoviesComponent(movies));
  } 
  // Rutas a páginas específicas
  else {
    switch (route) {
      case "#/":
        { let movies = await getMovies();
          fillContainer(buildMoviesComponent(movies));
        break; }
      case "#/movies":
        { 
          let movies = await getMovies();
          fillContainer(buildMoviesComponent(movies));
        break; }
      // Añadir más rutas según sea necesario
      default:
        console.log('404 page not found');
    }
  }
};


document.addEventListener("DOMContentLoaded", async () => {
  const menuDiv = document.querySelector("#menu");
  menuDiv.append(buildMenu());
  const containerDiv = document.querySelector("#container");
  
  window.location.hash = "#/";
  router(window.location.hash, containerDiv);
  window.addEventListener("hashchange", () => {
    router(window.location.hash, containerDiv);
  });
  
});
```

Ahora tocaría añadir el resto de rutas para otras búsquedas sobre paises o compañias. También hay que crear el enlace en los botones. Así en la vista, añadimos los eventos a los botones:

```javascript
divWrapper.querySelectorAll('button[data-genre]').forEach(button => button.addEventListener('click',()=>{
      window.location.hash = `#/movies/genre/${button.dataset.genre}`;
}));
```